# Merge Datasource

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import io , os , time ,requests,json,bs4
from time import sleep
import lxml.etree as xml
import pandas_datareader as pdr
from yahoo_finance import Share
import matplotlib.pyplot as plt
%matplotlib inline

# model
import pmdarima as pm

In [3]:
# init 
datapath = '/mnt/c/Users/poom/drive/01project/01cap_stone/04data/01stockprice'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [54]:
## import data
# Historical stockprice and technical indicator
sp500histprice = pd.read_csv(os.path.join(datapath,'histprice3.csv'))
# sentimental analysis
sentimental = pd.read_csv(os.path.join(datapath,'sentimental_analysis2.csv'))
# GG trends
ggtrend = pd.read_csv(os.path.join(datapath,'GGTrendallTimeMonthly.csv'))
# World indicies
indices = pd.read_csv(os.path.join(datapath,'General_data.csv'))

In [55]:
sp500histprice.head()

,Date,High,Low,Open,Close,Volume,Adj Close,ema10,ema20,ema75,macd,macd_signal_line,rsi,L14,H14,%K,%D,name
15,2015-01-22,28.117500,27.430000,27.565001,28.100000,215185600.0,25.597706,24.955144,25.045723,25.371346,-0.144608,-0.192319,54.163214,26.157499,28.312500,90.139232,63.747240,AAPL
16,2015-01-23,28.437500,27.882500,28.075001,28.245001,185859200.0,25.729790,25.095989,25.110872,25.380778,-0.072844,-0.168424,57.671228,26.157499,28.437500,91.557057,79.590968,AAPL
17,2015-01-26,28.590000,28.200001,28.434999,28.275000,222460000.0,25.757124,25.216195,25.172420,25.390682,-0.013608,-0.137461,66.442618,26.157499,28.590000,87.050342,89.582210,AAPL
18,2015-01-27,28.120001,27.257500,28.105000,27.285000,382274800.0,24.855284,25.150575,25.142216,25.376593,-0.038984,-0.117766,55.811122,26.299999,28.590000,43.013110,73.873503,AAPL
19,2015-01-28,29.530001,28.827499,29.407499,28.827499,585908400.0,26.260420,25.352365,25.248712,25.399851,0.053669,-0.083479,62.830907,26.299999,29.530001,78.250744,69.438065,AAPL


In [5]:
sentimental.head()

,ticker,date,compound
0,A,2020-05-22,0.0000
1,A,2020-05-23,-0.3753
2,A,2020-05-25,0.2500
3,A,2020-05-27,0.0000
4,A,2020-05-28,0.4462


In [6]:
ggtrend = ggtrend.drop('Unnamed: 0', axis = 1)
ggtrend.head()

,date,Name,GGTrendAllTimeMonthly
0,2004-01-01,AAPL,3
1,2004-02-01,AAPL,1
2,2004-03-01,AAPL,6
3,2004-04-01,AAPL,2
4,2004-05-01,AAPL,2


In [7]:
indices.head()

,Unnamed: 0,Date,d_Year,d_Month,d_Date,d_FirstDayOfMonth,d_DayInWeek,d_WeekInMonth,d_WeekInYear,COILOpen,COILHigh,COILLow,COILClose,COILAdj Close,COILVolume,DJIOpen,DJIHigh,DJILow,DJIClose,DJIAdj Close,DJIVolume,EURUSDOpen,EURUSDHigh,EURUSDLow,EURUSDClose,EURUSDAdj Close,EURUSDVolume,GOLDOpen,GOLDHigh,GOLDLow,GOLDClose,GOLDAdj Close,GOLDVolume,IXICOpen,IXICHigh,IXICLow,IXICClose,IXICAdj Close,IXICVolume,N225Open,N225High,N225Low,N225Close,N225Adj Close,N225Volume,NGFOpen,NGFHigh,NGFLow,NGFClose,NGFAdj Close,NGFVolume,NYAOpen,NYAHigh,NYALow,NYAClose,NYAAdj Close,NYAVolume,SILVEROpen,SILVERHigh,SILVERLow,SILVERClose,SILVERAdj Close,SILVERVolume,SP500Open,SP500High,SP500Low,SP500Close,SP500Adj Close,SP500Volume,USDCNYOpen,USDCNYHigh,USDCNYLow,USDCNYClose,USDCNYAdj Close,USDCNYVolume
0,0,2015-01-05,2015,1,5,0,0,2,2,52.610001,52.730000,49.680000,50.040001,50.040001,375782.0,17821.300781,17821.300781,17475.929688,17501.650391,17501.650391,116160000,1.195500,1.197590,1.188909,1.194643,1.194643,0.0,1180.300049,1206.900024,1180.099976,1203.900024,1203.900024,470.0,4700.339844,4702.770020,4641.459961,4652.569824,4652.569824,1794470000,17325.679688,17540.919922,17219.220703,17408.710938,17408.710938,116500.0,3.076,3.176,2.875,2.882,2.882,141783.0,10764.370117,10764.370117,10588.049805,10607.679688,10607.679688,3799120000,16.179001,16.179001,16.179001,16.179001,16.179001,0.0,2054.439941,2054.439941,2017.339966,2020.579956,2020.579956,3799120000,6.1961,6.2090,6.1961,6.1961,6.1961,0.0
1,1,2015-01-06,2015,1,6,0,1,2,2,50.000000,50.369999,47.549999,47.930000,47.930000,451642.0,17504.179688,17581.050781,17262.369141,17371.640625,17371.640625,101870000,1.193830,1.197000,1.188693,1.193902,1.193902,0.0,1203.500000,1220.000000,1203.500000,1219.300049,1219.300049,97.0,4666.850098,4667.330078,4567.589844,4592.740234,4592.740234,2167320000,17101.580078,17111.359375,16881.730469,16883.189453,16883.189453,166000.0,2.921,2.955,2.811,2.938,2.938,111338.0,10610.780273,10647.209961,10457.889648,10514.870117,10514.870117,4460110000,16.603001,16.603001,16.603001,16.603001,16.603001,2.0,2022.150024,2030.250000,1992.439941,2002.609985,2002.609985,4460110000,6.2084,6.2084,6.1988,6.2084,6.2084,0.0
2,2,2015-01-07,2015,1,7,0,2,2,2,48.000000,49.310001,46.830002,48.650002,48.650002,460083.0,17374.779297,17597.080078,17374.779297,17584.519531,17584.519531,91030000,1.187479,1.190000,1.180401,1.187536,1.187536,0.0,1219.199951,1219.199951,1210.599976,1210.599976,1210.599976,29.0,4626.839844,4652.720215,4613.899902,4650.470215,4650.470215,1957950000,16808.259766,16974.609375,16808.259766,16885.330078,16885.330078,138600.0,2.949,3.012,2.825,2.871,2.871,134826.0,10514.480469,10638.740234,10514.480469,10627.129883,10627.129883,3805480000,16.480000,16.549999,16.480000,16.510000,16.510000,9.0,2005.550049,2029.609985,2005.550049,2025.900024,2025.900024,3805480000,6.2020,6.2040,6.1976,6.2020,6.2020,0.0
3,3,2015-01-08,2015,1,8,0,3,2,2,48.779999,49.650002,47.730000,48.790001,48.790001,362081.0,17591.970703,17916.039063,17591.970703,17907.869141,17907.869141,114890000,1.183894,1.184806,1.175601,1.183600,1.183600,0.0,1207.000000,1215.699951,1206.300049,1208.400024,1208.400024,92.0,4689.540039,4741.379883,4688.020020,4736.189941,4736.189941,2105450000,17067.400391,17243.710938,17016.089844,17167.099609,17167.099609,140600.0,2.877,2.968,2.816,2.927,2.927,147797.0,10686.000000,10809.459961,10686.000000,10800.540039,10800.540039,3934010000,16.351000,16.351000,16.351000,16.351000,16.351000,0.0,2030.609985,2064.080078,2030.609985,2062.139893,2062.139893,3934010000,6.2040,6.2102,6.2035,6.2040,6.2040,0.0
4,4,2015-01-09,2015,1,9,0,4,2,2,48.919998,49.610001,47.160000,48.360001,48.360001,421007.0,17911.019531,17915.320313,17686.089844,17737.369141,17737.369141,93390000,1.179426,1.183830,1.176831,1.179607,1.179607,0.0,1211.099976,1220.199951,1210.300049,1216.000000,1216.000000,43.0,4744.470215,4744.709961,4681.240234,4704.069824,4704.069824,1715830000,17318.740234,17342.650391,17129.529297,1719

In [44]:
indices = pd.concat([indices[['Date']],indices.iloc[:,9:]],axis = 1)
indices.head()

,Date,COILOpen,COILHigh,COILLow,COILClose,COILAdj Close,COILVolume,DJIOpen,DJIHigh,DJILow,DJIClose,DJIAdj Close,DJIVolume,EURUSDOpen,EURUSDHigh,EURUSDLow,EURUSDClose,EURUSDAdj Close,EURUSDVolume,GOLDOpen,GOLDHigh,GOLDLow,GOLDClose,GOLDAdj Close,GOLDVolume,IXICOpen,IXICHigh,IXICLow,IXICClose,IXICAdj Close,IXICVolume,N225Open,N225High,N225Low,N225Close,N225Adj Close,N225Volume,NGFOpen,NGFHigh,NGFLow,NGFClose,NGFAdj Close,NGFVolume,NYAOpen,NYAHigh,NYALow,NYAClose,NYAAdj Close,NYAVolume,SILVEROpen,SILVERHigh,SILVERLow,SILVERClose,SILVERAdj Close,SILVERVolume,SP500Open,SP500High,SP500Low,SP500Close,SP500Adj Close,SP500Volume,USDCNYOpen,USDCNYHigh,USDCNYLow,USDCNYClose,USDCNYAdj Close,USDCNYVolume
0,2015-01-05,52.610001,52.730000,49.680000,50.040001,50.040001,375782.0,17821.300781,17821.300781,17475.929688,17501.650391,17501.650391,116160000,1.195500,1.197590,1.188909,1.194643,1.194643,0.0,1180.300049,1206.900024,1180.099976,1203.900024,1203.900024,470.0,4700.339844,4702.770020,4641.459961,4652.569824,4652.569824,1794470000,17325.679688,17540.919922,17219.220703,17408.710938,17408.710938,116500.0,3.076,3.176,2.875,2.882,2.882,141783.0,10764.370117,10764.370117,10588.049805,10607.679688,10607.679688,3799120000,16.179001,16.179001,16.179001,16.179001,16.179001,0.0,2054.439941,2054.439941,2017.339966,2020.579956,2020.579956,3799120000,6.1961,6.2090,6.1961,6.1961,6.1961,0.0
1,2015-01-06,50.000000,50.369999,47.549999,47.930000,47.930000,451642.0,17504.179688,17581.050781,17262.369141,17371.640625,17371.640625,101870000,1.193830,1.197000,1.188693,1.193902,1.193902,0.0,1203.500000,1220.000000,1203.500000,1219.300049,1219.300049,97.0,4666.850098,4667.330078,4567.589844,4592.740234,4592.740234,2167320000,17101.580078,17111.359375,16881.730469,16883.189453,16883.189453,166000.0,2.921,2.955,2.811,2.938,2.938,111338.0,10610.780273,10647.209961,10457.889648,10514.870117,10514.870117,4460110000,16.603001,16.603001,16.603001,16.603001,16.603001,2.0,2022.150024,2030.250000,1992.439941,2002.609985,2002.609985,4460110000,6.2084,6.2084,6.1988,6.2084,6.2084,0.0
2,2015-01-07,48.000000,49.310001,46.830002,48.650002,48.650002,460083.0,17374.779297,17597.080078,17374.779297,17584.519531,17584.519531,91030000,1.187479,1.190000,1.180401,1.187536,1.187536,0.0,1219.199951,1219.199951,1210.599976,1210.599976,1210.599976,29.0,4626.839844,4652.720215,4613.899902,4650.470215,4650.470215,1957950000,16808.259766,16974.609375,16808.259766,16885.330078,16885.330078,138600.0,2.949,3.012,2.825,2.871,2.871,134826.0,10514.480469,10638.740234,10514.480469,10627.129883,10627.129883,3805480000,16.480000,16.549999,16.480000,16.510000,16.510000,9.0,2005.550049,2029.609985,2005.550049,2025.900024,2025.900024,3805480000,6.2020,6.2040,6.1976,6.2020,6.2020,0.0
3,2015-01-08,48.779999,49.650002,47.730000,48.790001,48.790001,362081.0,17591.970703,17916.039063,17591.970703,17907.869141,17907.869141,114890000,1.183894,1.184806,1.175601,1.183600,1.183600,0.0,1207.000000,1215.699951,1206.300049,1208.400024,1208.400024,92.0,4689.540039,4741.379883,4688.020020,4736.189941,4736.189941,2105450000,17067.400391,17243.710938,17016.089844,17167.099609,17167.099609,140600.0,2.877,2.968,2.816,2.927,2.927,147797.0,10686.000000,10809.459961,10686.000000,10800.540039,10800.540039,3934010000,16.351000,16.351000,16.351000,16.351000,16.351000,0.0,2030.609985,2064.080078,2030.609985,2062.139893,2062.139893,3934010000,6.2040,6.2102,6.2035,6.2040,6.2040,0.0
4,2015-01-09,48.919998,49.610001,47.160000,48.360001,48.360001,421007.0,17911.019531,17915.320313,17686.089844,17737.369141,17737.369141,93390000,1.179426,1.183830,1.176831,1.179607,1.179607,0.0,1211.099976,1220.199951,1210.300049,1216.000000,1216.000000,43.0,4744.470215,4744.709961,4681.240234,4704.069824,4704.069824,1715830000,17318.740234,17342.650391,17129.529297,17197.730469,17197.730469,155200.0,2.959,2.990,2.888,2.946,2.946,107333.0,10811.820313,10811.820313,10680.530273,10711.410156,10711.410156,3364140000,16.386000,16.386000,16.386000,16.386000

In [70]:
df_final = sp500histprice.copy()
df_final = pd.merge(df_final,indices, how='left',left_on=['Date'], right_on = ['Date'])
df_final = pd.merge(df_final,sentimental, how='left',left_on=['Date','name'], right_on = ['date','ticker']).drop(['ticker','date'],axis =1)
df_final = pd.merge(df_final,ggtrend, how='left',left_on=['Date','name'], right_on = ['date','Name']).drop(['date','Name'],axis =1)
df_final.head()

,Date,High,Low,Open,Close,Volume,Adj Close,ema10,ema20,ema75,macd,macd_signal_line,rsi,L14,H14,%K,%D,name,COILOpen,COILHigh,COILLow,COILClose,COILAdj Close,COILVolume,DJIOpen,DJIHigh,DJILow,DJIClose,DJIAdj Close,DJIVolume,EURUSDOpen,EURUSDHigh,EURUSDLow,EURUSDClose,EURUSDAdj Close,EURUSDVolume,GOLDOpen,GOLDHigh,GOLDLow,GOLDClose,GOLDAdj Close,GOLDVolume,IXICOpen,IXICHigh,IXICLow,IXICClose,IXICAdj Close,IXICVolume,N225Open,N225High,N225Low,N225Close,N225Adj Close,N225Volume,NGFOpen,NGFHigh,NGFLow,NGFClose,NGFAdj Close,NGFVolume,NYAOpen,NYAHigh,NYALow,NYAClose,NYAAdj Close,NYAVolume,SILVEROpen,SILVERHigh,SILVERLow,SILVERClose,SILVERAdj Close,SILVERVolume,SP500Open,SP500High,SP500Low,SP500Close,SP500Adj Close,SP500Volume,USDCNYOpen,USDCNYHigh,USDCNYLow,USDCNYClose,USDCNYAdj Close,USDCNYVolume,compound,GGTrendAllTimeMonthly
0,2015-01-22,28.117500,27.430000,27.565001,28.100000,215185600.0,25.597706,24.955144,25.045723,25.371346,-0.144608,-0.192319,54.163214,26.157499,28.312500,90.139232,63.747240,AAPL,47.349998,49.090000,45.869999,46.310001,46.310001,426829.0,17557.289063,17840.890625,17482.539063,17813.980469,17813.980469,111980000.0,1.161697,1.164289,1.141400,1.161535,1.161535,0.0,1293.199951,1302.099976,1283.500000,1300.699951,1300.699951,131.0,4690.930176,4752.600098,4644.569824,4750.399902,4750.399902,1.996550e+09,17306.640625,17355.740234,17229.210938,17329.019531,17329.019531,121900.0,3.000,3.048,2.766,2.835,2.835,169450.0,10753.580078,10869.000000,10699.730469,10859.349609,10859.349609,4.176050e+09,18.346001,18.346001,18.346001,18.346001,18.346001,0.0,2034.300049,2064.620117,2026.380005,2063.149902,2063.149902,4.176050e+09,6.2017,6.2020,6.1986,6.2017,6.2017,0.0,NaN,NaN
1,2015-01-23,28.437500,27.882500,28.075001,28.245001,185859200.0,25.729790,25.095989,25.110872,25.380778,-0.072844,-0.168424,57.671228,26.157499,28.437500,91.557057,79.590968,AAPL,46.630001,47.759998,45.209999,45.590000,45.590000,374289.0,17812.500000,17812.500000,17667.529297,17672.599609,17672.599609,97110000.0,1.134649,1.137398,1.112180,1.134675,1.134675,0.0,1299.800049,1299.800049,1292.599976,1292.599976,1292.599976,23.0,4748.189941,4771.180176,4737.950195,4757.879883,4757.879883,1.653830e+09,17520.630859,17532.060547,17460.759766,17511.750000,17511.750000,132500.0,2.858,2.994,2.855,2.986,2.986,99068.0,10854.230469,10854.230469,10784.570313,10788.330078,10788.330078,3.573560e+09,18.315001,18.355000,18.284000,18.284000,18.284000,20.0,2062.979980,2062.979980,2050.540039,2051.820068,2051.820068,3.573560e+09,6.1990,6.2179,6.1990,6.1990,6.1990,0.0,NaN,NaN
2,2015-01-26,28.590000,28.200001,28.434999,28.275000,222460000.0,25.757124,25.216195,25.172420,25.390682,-0.013608,-0.137461,66.442618,26.157499,28.590000,87.050342,89.582210,AAPL,45.200001,46.410000,44.349998,45.150002,45.150002,310638.0,17668.109375,17696.359375,17567.599609,17678.699219,17678.699219,87220000.0,1.114082,1.128399,1.114082,1.112545,1.112545,0.0,1289.500000,1290.599976,1279.400024,1279.400024,1279.400024,26.0,4752.359863,4774.180176,4734.200195,4771.759766,4771.759766,1.717510e+09,17285.710938,17471.939453,17285.710938,17468.519531,17468.519531,111600.0,2.897,2.955,2.841,2.881,2.881,62580.0,10787.299805,10848.040039,10748.730469,10847.150391,10847.150391,3.465760e+09,17.966999,17.966999,17.966999,17.966999,17.966999,0.0,2050.419922,2057.620117,2040.969971,2057.090088,2057.090088,3.465760e+09,6.2179,6.2458,6.2179,6.2179,6.2179,0.0,NaN,NaN
3,2015-01-27,28.120001,27.257500,28.105000,27.285000,382274800.0,24.855284,25.150575,25.142216,25.376593,-0.038984,-0.117766,55.811122,26.299999,28.590000,43.013110,73.873503,AAPL,45.130001,46.549999,44.810001,46.230000,46.230000,313141.0,17638.529297,17638.529297,17288.310547,17387.210938,17387.210938,135940000.0,1.125100,1.142000,1.122600,1.124771,1.124771,0.0,1277.199951,1291.699951,1277.199951,1291.699951,1291.699951,20.0,4698.229980,4721.870117,4659.830078,4681.500000,4681.500000,1.954160e+09,17649.400391,17768.410156,17633.470703,17768.300781,17768.30078

In [72]:
df_final = df_final.drop('Close', axis = 1)
df_final = df_final.fillna({'GGTrendAllTimeMonthly':0,'compound':0})
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 739464 entries, 0 to 739463
Data columns (total 85 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Date                   739464 non-null  object 
 1   High                   739464 non-null  float64
 2   Low                    739464 non-null  float64
 3   Open                   739464 non-null  float64
 4   Volume                 739464 non-null  float64
 5   Adj Close              739464 non-null  float64
 6   ema10                  739464 non-null  float64
 7   ema20                  739464 non-null  float64
 8   ema75                  739464 non-null  float64
 9   macd                   739464 non-null  float64
 10  macd_signal_line       739464 non-null  float64
 11  rsi                    739464 non-null  float64
 12  L14                    739464 non-null  float64
 13  H14                    739464 non-null  float64
 14  %K                     739464 non-nu

In [73]:
df_final.isna().sum()

Date                         0
High                         0
Low                          0
Open                         0
Volume                       0
Adj Close                    0
ema10                        0
ema20                        0
ema75                        0
macd                         0
macd_signal_line             0
rsi                          0
L14                          0
H14                          0
%K                           0
%D                           0
name                         0
COILOpen                 38072
COILHigh                 38072
COILLow                  38072
COILClose                38072
COILAdj Close            38072
COILVolume               38072
DJIOpen                  38072
DJIHigh                  38072
DJILow                   38072
DJIClose                 38072
DJIAdj Close             38072
DJIVolume                38072
EURUSDOpen               38072
EURUSDHigh               38072
EURUSDLow                38072
EURUSDCl

In [74]:
df_final.to_csv(os.path.join(datapath,'df_final.csv'))

## Load data

In [12]:
df_final= pd.read_csv(os.path.join(datapath,'df_final.csv'))
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739464 entries, 0 to 739463
Data columns (total 86 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             739464 non-null  int64  
 1   Date                   739464 non-null  object 
 2   High                   739464 non-null  float64
 3   Low                    739464 non-null  float64
 4   Open                   739464 non-null  float64
 5   Volume                 739464 non-null  float64
 6   Adj Close              739464 non-null  float64
 7   ema10                  739464 non-null  float64
 8   ema20                  739464 non-null  float64
 9   ema75                  739464 non-null  float64
 10  macd                   739464 non-null  float64
 11  macd_signal_line       739464 non-null  float64
 12  rsi                    739464 non-null  float64
 13  L14                    739464 non-null  float64
 14  H14                    739464 non-nu

In [14]:
# df_final = df_final.drop('Unnamed: 0',axis = 1)
# df_final.fillna(method='ffill',inplace=True)
# df_final.isna().sum().sum()
df_final.to_csv(os.path.join(datapath,'df_final2.csv'),index=False)

In [ ]:
df_final= pd.read_csv(os.path.join(datapath,'df_final2.csv'))